# Welcome to Tutorial 5!


---

####  **This tutorial shall explore Vulnerability Detection in [Episimmer](https://github.com/healthbadge/episimmer/tree/master).**



##### Let us perform some initialization first. 

In [3]:
!pip3 install pyvis==0.1.9
!pip3 install episimmer --no-dependencies
!git clone https://github.com/healthbadge/episimmer.git
%cd episimmer
from google.colab.patches import cv2, cv2_imshow
from google.colab import _message
import IPython
import json

     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 1.9 MB 11.1 MB/s 
     |████████████████████████████████| 38.1 MB 1.1 MB/s 
     |████████████████████████████████| 10.3 MB 43.3 MB/s 
     |████████████████████████████████| 15.7 MB 32.0 MB/s 
     |████████████████████████████████| 4.3 MB 42.8 MB/s 
     |████████████████████████████████| 247 kB 66.5 MB/s 
     |████████████████████████████████| 1.1 MB 55.7 MB/s 
     |████████████████████████████████| 98 kB 9.2 MB/s 
     |████████████████████████████████| 132 kB 50.9 MB/s 
     |████████████████████████████████| 793 kB 60.8 MB/s 
     |████████████████████████████████| 381 kB 64.5 MB/s 
     |████████████████████████████████| 1.1 MB 48.8 MB/s 
     |████████████████████████████████| 1.6 MB 29.8 MB/s 
     |████████████████████████████████| 102 kB 13.1 MB/s 
     |████████████████████████████████| 59 kB 7.4 MB/s 
     |████████████████████████████████| 100 kB 10.6 MB/s 


Cloning into 'episimmer'...
remote: Enumerating objects: 6497, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 6497 (delta 37), reused 41 (delta 16), pack-reused 6424
Receiving objects: 100% (6497/6497), 49.40 MiB | 35.65 MiB/s, done.
Resolving deltas: 100% (4406/4406), done.
/content/episimmer
     |████████████████████████████████| 131 kB 5.2 MB/s 
     |████████████████████████████████| 130 kB 46.4 MB/s 
     |████████████████████████████████| 1.8 MB 73.4 MB/s 
     |████████████████████████████████| 40 kB 6.3 MB/s 
     |████████████████████████████████| 280 kB 69.8 MB/s 
     |████████████████████████████████| 428 kB 63.7 MB/s 
     |████████████████████████████████| 1.1 MB 58.4 MB/s 
     |████████████████████████████████| 87 kB 8.1 MB/s 


# Vulnerability Detection

---

Vulnerability Detection refers to finding vulnerabilities in the system that highly affect or are highly affected by
disease spread. A major part of decision support is detecting vulnerabilities in the ecosystem and taking appropriate
actions to control the disease spread. For detection of any type of vulnerabilities in the system, we have simply used
the [Monte Carlo method](https://en.wikipedia.org/wiki/Monte_Carlo_method#:~:text=Monte%20Carlo%20methods%2C%20or%20Monte,might%20be%20deterministic%20in%20principle).

In Episimmer, there are two broad classes of Vulnerability Detection -
Agent-based Vulnerability Detection and Event-based Vulnerability Detection. We will only be discussing Agent-based Vulnerability Detection in this Tutorial.

In Agent-based VD, we have two types : 

* **Vulnerable Agents** : Agents who are most likely to reach a certain disease state. For example ‘agents most likely to die’ or ‘agents most likely to get infected’ or ‘agents most likely to be hospitalised’.
* **Agent Vulnerabilities** : The Agent on removal or constrained for a duration of time from the ecosystem, lead to the largest reduction in risk or size of the epidemic. This can be viewed as disconnecting parts of the system to stop spread of a disease.

While running a VD module, every agent has an associated agent score. Vulnerable agents will have higher scores when they seem to be getting into the vulnerable states specified, more than the other agents. Thus, a higher score indicates a relative more vulnerable agent.
Agent vulnerability scores correspond to the agents influence when removed from the environment, considering the agent vulnerability states specified. For example, if we consider a 'Dead' disease state as the agent vulnerability state, when we remove an agent, say Agent A,
his absence in the environment might be critical to reducing deaths.


We shall discuss and run a single algorithm of Vulnerable Agents and Agent Vulnerabilities in this Tutorial.

VD is run in Episimmer with the help of two files - vd_config.txt and parameter.json files.

These two files will help us configure the parameters of the VD run.

The v**d_config file** has a specific format and is shown below.

```text
VD Target <>
VD Algorithm <>
Algorithm Parameter File <>
Pre Processing <>
Post Processing <>
Output Mode <>
```

* VD Target field refers to whether the Vulnerability Detection (VD) algorithm must target Agents or Events. Thus, the target
value can be Agent or Event.

* VD Algorithm field refers to the Algorithm that must be used for Vulnerability Detection. Do note that each target has different
algorithms available to it. For Agent Target, algorithms could be - SimpleVulnerableAgent, EarlyVulnerableAgent, SimpleAgentVulnerability,
ChunkAgentVulnerability and BanditAlgos. For Event Target, only one algorithm can be used - SimpleEventVulnerability.

* Algorithm Parameter File field contains the parameters file which contains the parameter values required by the algorithm. The name of
the Parameters file must be given here.

* Pre Processing and Post Processing fields refer to operations done before and after the VD simulations. You can leave
both of these fields blank for now.

* Output Mode refers to what should be printed after the completion of the entire process of VD (after Post processing). Currently,
there is only a single Output Mode - Default.

The **parameters file** contains the parameters of the specific Vulnerability Detection algorithm to be used. It must be a json file
and the name of this file must be passed in the *Algorithm Parameter File* field in the vd_config.txt file. An example is shown below

```json
{
    "states":["Infected","Recovered"],
    "num_runs":100,
}
```
All algorithms have the states and num_runs parameters. States refers to the states in the simulation that are infectious or post-infectious while
num_runs refers to the total number of simulation runs. In most disease models, you should have a "Susceptible" state, and at the end of a
simulation, the agents belonging to this state were the only ones that were not affected by the disease. This would mean that all the other agents
were infected at some point in time. This is why you must choose all the agent states that are considered to be either infected or passed an infected
state at some point in time. For example, if the SIR model is used, the VD states you should choose are both the Infected(I) and Recovered(R) states.



### Simple Vulnerable Agents

This algorithm scores the agent based on the number of times he was infected in multiple simulations.

$$Agent\ Score = \sum_{i=0}^{N-1}{ 1[Agent\ Infected]}$$

where N refers to the total number of simulation runs.

### Simple Agent Vulnerability

This algorithm scores the agent based on the severity of outbreak after the agent's removal. Multiple simulations are run by randomly selecting and removing an agent and running the simulation. The agent score is calculated by finding the running average of scores for each run.

$$Agent\ Score = 1 - \frac{Total\ Agents\ Infected}{Total\ Agents}$$

In [4]:
# Some helper functions
def read_file(file_name):
  with open(file_name) as f:
    print(f.read())

### Star Graph 

In [5]:
!mkdir StarG
!cp -a examples/Vulnerability_Detection/Star_Graph/. StarG/

In [6]:
!python3 scripts/viz/view_interaction_graph.py -n 7 -f StarG/interactions_list.txt
IPython.display.HTML(filename='StarG/interactions_list.html')

As shown, the star graph consists of a single node ('0') connected to all other nodes. In this case, our largest Vulnerable agent and highest agent vulnerability will be the central node. 

Let us manipulate the vd_config.txt and parameter.json file to print Vulnerable agents and Agent Vulnerabilities.

First, let us find the top Vulnerable agents.

In [7]:
vd_config_str = "VD Target <Agent>\n\
VD Algorithm <SimpleVulnerableAgent>\n\
Algorithm Parameter File <parameter.json>\n\
Pre Processing <>\n\
Post Processing <>\n\
Output Mode <Default>"

with open("StarG/vd_config.txt", "w") as f:
  f.write(vd_config_str)

In [8]:
jdict = {
    "states":["Infected","Recovered"],
    "num_runs":1000
}

json_object = json.dumps(jdict, indent = 4)

with open("StarG/parameter.json", "w") as outfile:
    outfile.write(json_object)

In [9]:
!python3 episimmer/main.py StarG -np -vul

Iteration running :  100
Iteration running :  200
Iteration running :  300
Iteration running :  400
Iteration running :  500
Iteration running :  600
Iteration running :  700
Iteration running :  800
Iteration running :  900
Iteration running :  1000
{'0': 973.0, '6': 881.0, '3': 871.0, '2': 864.0, '1': 860.0, '4': 856.0, '5': 855.0, '9': 499.0, '7': 490.0, '8': 469.0}
{'8': 469.0, '7': 490.0, '9': 499.0, '5': 855.0, '4': 856.0, '1': 860.0, '2': 864.0, '3': 871.0, '6': 881.0, '0': 973.0}


Our highest score is expected as it is for Agent '0' which is our central node. 

Now, lets us find the top Agent Vulnerabilities

In [10]:
vd_config_str = "VD Target <Agent>\n\
VD Algorithm <SimpleAgentVulnerability>\n\
Algorithm Parameter File <parameter.json>\n\
Pre Processing <>\n\
Post Processing <>\n\
Output Mode <Default>"

with open("StarG/vd_config.txt", "w") as f:
  f.write(vd_config_str)

In [11]:
jdict = {
    "states":["Infected","Recovered"],
    "num_runs":1000,
    "num_agents_to_remove":1
}

json_object = json.dumps(jdict, indent = 4)

with open("StarG/parameter.json", "w") as outfile:
    outfile.write(json_object)

In [12]:
!python3 episimmer/main.py StarG -np -vul

Iteration running :  100
Iteration running :  200
Iteration running :  300
Iteration running :  400
Iteration running :  500
Iteration running :  600
Iteration running :  700
Iteration running :  800
Iteration running :  900
Iteration running :  1000
{'0': 0.541, '2': 0.2801, '1': 0.2691, '6': 0.2567, '5': 0.2524, '4': 0.2486, '3': 0.2341, '9': 0.2145, '7': 0.2037, '8': 0.1848}
{'8': 0.1848, '7': 0.2037, '9': 0.2145, '3': 0.2341, '4': 0.2486, '5': 0.2524, '6': 0.2567, '1': 0.2691, '2': 0.2801, '0': 0.541}


Once again we get the highest Agent Vulnerability as node '0' which is expected.

### Multi-Cycle Graph

In [13]:
!mkdir MCycleG
!cp -a examples/Vulnerability_Detection/Multi_Cycle_Graph/. MCycleG/

In [14]:
!python3 scripts/viz/view_interaction_graph.py -n 30 -f MCycleG/interactions_list.csv
IPython.display.HTML(filename='MCycleG/interactions_list.html')

This Multi-Cycle graph is connected by three major nodes - '0', '10' and '20'. 

Let us find the top Vulnerable agents first

In [15]:
vd_config_str = "VD Target <Agent>\n\
VD Algorithm <SimpleVulnerableAgent>\n\
Algorithm Parameter File <parameter.json>\n\
Pre Processing <>\n\
Post Processing <>\n\
Output Mode <Default>"

with open("MCycleG/vd_config.txt", "w") as f:
  f.write(vd_config_str)

In [16]:
jdict = {
    "states":["Infected","Recovered"],
    "num_runs":1000,
}

json_object = json.dumps(jdict, indent = 4)

with open("MCycleG/parameter.json", "w") as outfile:
    outfile.write(json_object)

In [17]:
!python3 episimmer/main.py MCycleG -np -vul

Iteration running :  100
Iteration running :  200
Iteration running :  300
Iteration running :  400
Iteration running :  500
Iteration running :  600
Iteration running :  700
Iteration running :  800
Iteration running :  900
Iteration running :  1000
{'0': 751.0, '20': 747.0, '10': 745.0, '9': 647.0, '1': 644.0, '11': 632.0, '19': 623.0, '21': 618.0, '29': 608.0, '8': 577.0}
{'83': 82.0, '37': 83.0, '56': 83.0, '84': 85.0, '64': 86.0, '79': 86.0, '89': 86.0, '54': 87.0, '44': 88.0, '78': 88.0}


Our top 3 Vulnerable agents are - '0', '10' and '20'.

Now let us find the top agent vulnerabilities

In [18]:
vd_config_str = "VD Target <Agent>\n\
VD Algorithm <SimpleAgentVulnerability>\n\
Algorithm Parameter File <parameter.json>\n\
Pre Processing <>\n\
Post Processing <>\n\
Output Mode <Default>"

with open("MCycleG/vd_config.txt", "w") as f:
  f.write(vd_config_str)

Since we have three central nodes, we shall set num_agents_to_remove to 3.

In [19]:
jdict = {
    "states":["Infected","Recovered"],
    "num_runs":5000,
    "num_agents_to_remove":3
}


json_object = json.dumps(jdict, indent = 4)

with open("MCycleG/parameter.json", "w") as outfile:
    outfile.write(json_object)

In [20]:
!python3 episimmer/main.py MCycleG -np -vul

Iteration running :  500
Iteration running :  1000
Iteration running :  1500
Iteration running :  2000
Iteration running :  2500
Iteration running :  3000
Iteration running :  3500
Iteration running :  4000
Iteration running :  4500
Iteration running :  5000
{'20': 0.8073, '0': 0.8062, '10': 0.803, '17': 0.7959, '9': 0.7936, '88': 0.7855, '14': 0.7845, '13': 0.7832, '18': 0.783, '74': 0.7829}
{'66': 0.7516, '83': 0.7539, '63': 0.7559, '99': 0.7577, '96': 0.7583, '93': 0.7586, '34': 0.7587, '37': 0.7599, '54': 0.76, '92': 0.7608}


This is quite interesting as we have three central nodes, instead of just 1. That is why removal of just one of the three nodes will not help as much as removing all three. 

### Complex Graph

In [21]:
!mkdir ComplexG
!cp -a examples/Vulnerability_Detection/Complex_Graph/. ComplexG/

In [22]:
!python3 scripts/viz/view_interaction_graph.py -n 13 -f ComplexG/interactions_list.txt
IPython.display.HTML(filename='ComplexG/interactions_list.html')

This Complex graph is worth showing as the highest agent vulnerability need not be a node that is highly connected. Let us run the VD algorithm first.

In [23]:
vd_config_str = "VD Target <Agent>\n\
VD Algorithm <SimpleAgentVulnerability>\n\
Algorithm Parameter File <parameter.json>\n\
Pre Processing <>\n\
Post Processing <>\n\
Output Mode <Default>"

with open("ComplexG/vd_config.txt", "w") as f:
  f.write(vd_config_str)

In [24]:
jdict = {
    "states":["Infected","Recovered"],
    "num_runs":1000,
    "num_agents_to_remove":1
}

json_object = json.dumps(jdict, indent = 4)

with open("ComplexG/parameter.json", "w") as outfile:
    outfile.write(json_object)

In [25]:
!python3 episimmer/main.py ComplexG -np -vul

Iteration running :  100
Iteration running :  200
Iteration running :  300
Iteration running :  400
Iteration running :  500
Iteration running :  600
Iteration running :  700
Iteration running :  800
Iteration running :  900
Iteration running :  1000
{'8': 0.221, '9': 0.1909, '0': 0.1714, '4': 0.1544, '6': 0.1351, '7': 0.1338, '1': 0.1149, '5': 0.1104, '3': 0.1082, '12': 0.106}
{'10': 0.0432, '2': 0.0847, '11': 0.1039, '12': 0.106, '3': 0.1082, '5': 0.1104, '1': 0.1149, '7': 0.1338, '6': 0.1351, '4': 0.1544}


We get the highest agent vulnerability as node '8' because node '8' is responsible for disconnecting the two largest sections of the network. Do note that the graph strcuture is not the only reason. The speed with which the disease spreads and also the initial innfection in the population are big factors in determining the vulnerabilities.  

### Disclaimer : These colab tutorials are only for reference and they are in no way representative of the entire functionality of Episimmer. Please look at the [Documentation](https://episimmer.readthedocs.io/en/latest/) for more details.